**Import libraries and dependents**

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

**Scrape table from wiki page**

In [4]:
WIKI_URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
req = requests.get(WIKI_URL)
soup = BeautifulSoup(req.content,'lxml')
table = soup.find_all('table')[0]
df = pd.read_html(str(table))[0]
df.drop([0], inplace=True)
df.columns=['PostalCode','Borough','Neighborhood']
df.head()

,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


**Clean "Not assigned"**

In [5]:
df=df[df['Borough']!="Not assigned"].reset_index(drop=True)
for i in range(0,len(df)):
    if df.loc[i,'Neighborhood']=='Not assigned':
        df.loc[i,'Neighborhood']=df.loc[i,'Borough']
df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


**Group data by Postal Code and Borough**

In [6]:
grouped=df.groupby(['PostalCode','Borough'])
df_grouped=grouped['Neighborhood'].sum().reset_index()[['PostalCode','Borough']] 
df_grouped.head(5) # A frame with distinct Postal Code and Borough

,PostalCode,Borough
0,M1B,Scarborough
1,M1C,Scarborough
2,M1E,Scarborough
3,M1G,Scarborough
4,M1H,Scarborough


**Joining neighborhoods having the same Postal Code and Borough** 

In [39]:
df_grouped['Neighborhood']=df_grouped['Borough']  # initialize a new column for neighborhoods
for i in range(0,len(df_grouped)):
    pb=tuple(df_grouped.loc[i,['PostalCode','Borough']])
    mylist=grouped.get_group(pb)['Neighborhood']
    mystr=', '.join(mylist)
    df_grouped.loc[i,'Neighborhood']=mystr
df_grouped.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


**Data shape**

In [41]:
df_grouped.shape

(103, 3)